# Exploring AI Mentions in Earnings Calls and Building Thematic Portfolios

## Introduction

In recent years, artificial intelligence (AI) has emerged as a central pillar of innovation across various industries. As companies increasingly emphasize AI in their business strategies, earnings calls have become an important source of insights for investors seeking to understand corporate priorities and market trends.

This article examines how the frequency and sentiment of AI mentions in earnings calls have evolved over time and whether these patterns can indicate a company's long-term growth potential. To uncover these insights, we conducted an exploratory analysis of earnings call transcripts from companies across diverse sectors leveraging LSEG MarketPsych Transcripts Analytics (MTA)  API. Additionally, we developed thematic portfolios based on these insights and tracked their historical performance to assess whether investing in AI-focused companies can yield significant returns.

The article is structured into three sections: setting up the dataset, conducting exploratory analysis, and building thematic portfolios.

## Import relevent libraries

To start, we first install and import the necessary packages. We use the [LSEG Data Libraries](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python) to retrieve the company universe for the analyis and [LSEG MarketPsych Transcripts Analytics (MTA)](https://www.marketpsych.com/transcripts/guide/overview) API for earnings transcripts. The code is built using Python 3.9. The prerequisite packages are imported as shown below. Please make sure you have them installed.

In [1]:
import requests
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import lseg.data as ld
from lseg.data.discovery import convert_symbols, SymbolTypes
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans

ld.get_config()["http.request-timeout"] = 60
pd.options.display.max_columns = None

## Section 1: Setting up the dataset

### Getting the company universe for the analysis

The first step is to define the universe of companies for which we will request transcript data from MTA. For a robust analysis, it's important to include a large number of companies across various economic sectors. Therefore, we have chosen the constituents of the [Russell 1000]((https://www.lseg.com/en/ftse-russell/indices/russell-us/russell-1000)) index as our universe, which encompasses all large-cap companies in the US.

Since our primary objective is to conduct an exploratory analysis rather than develop a comprehensive portfolio construction strategy, we will use the current list of Russell 1000 constituents. For a more in-depth analysis of portfolio performance, it is advisable to track changes in index constituents over time (e.g., each quarter). In this regard, [this article](https://developers.lseg.com/en/article-catalog/article/building-historical-index-constituents) provides useful functions for retrieving historical index constituent data.

To begin, we open an API session and request the data for the universe of companies using LD Libraries for Python. Additionally, to implement also a sectoral analysis, we request information on the economic sectors of these companies.

In [2]:
ld.open_session()

<lseg.data.session.Definition object at 0x31d920e80 {name='workspace'}>

In [3]:
universe = ld.get_data('0#.RUI', 'TR.TRBCEconomicSector')
universe

,Instrument,TRBC Economic Sector Name
0,AIZ.N,Financials
1,GME.N,Consumer Cyclicals
2,KEYS.N,Technology
3,QCOM.OQ,Technology
4,KMPR.N,Financials
...,...,...
1005,NLY.N,Real Estate
1006,MA.N,Technology
1007,BIIB.OQ,Healthcare
1008,SYF.N,Financials


The request above returned the RICs of the FTSE 1000 contituent companies alongside with the economic sectors they represent. To request the transcripts data via MTA though, we need to use company perm_ids, therefore we leverage symbology conversion of the LD Libraries to get the perm_ids of our universe.

In [4]:
universe_rics = universe['Instrument'].to_list()

universe_perm_ids = convert_symbols(universe_rics, 
    from_symbol_type=SymbolTypes.RIC,
    to_symbol_types=[SymbolTypes.OA_PERM_ID],
)

universe_perm_ids

,DocumentTitle,IssuerOAPermID
AIZ.N,"Assurant Inc, Ordinary Share, New York Stock E...",4295899478
GME.N,"GameStop Corp, Ordinary Share, Class A, New Yo...",4295902037
KEYS.N,"Keysight Technologies Inc, Ordinary Share, New...",5041978269
QCOM.OQ,"Qualcomm Inc, Ordinary Share, NASDAQ Stock Exc...",4295907706
KMPR.N,"Kemper Corp, Ordinary Share, New York Stock Ex...",4295908262
...,...,...
NLY.N,"Annaly Capital Management Inc, Ordinary Share,...",4295903438
MA.N,"Mastercard Inc, Ordinary Share, Class A, New Y...",4295902520
BIIB.OQ,"Biogen Inc, Ordinary Share, NASDAQ Stock Excha...",4295906748
SYF.N,"Synchrony Financial, Ordinary Share, New York ...",5042214022


Now let's merge the requested perm_ids with our initial universe.

In [5]:
universe_perm_ids.drop_duplicates(subset = 'IssuerOAPermID', inplace =True)
universe = universe.merge(universe_perm_ids, left_on='Instrument', right_index=True)
universe

,Instrument,TRBC Economic Sector Name,DocumentTitle,IssuerOAPermID
0,AIZ.N,Financials,"Assurant Inc, Ordinary Share, New York Stock E...",4295899478
1,GME.N,Consumer Cyclicals,"GameStop Corp, Ordinary Share, Class A, New Yo...",4295902037
2,KEYS.N,Technology,"Keysight Technologies Inc, Ordinary Share, New...",5041978269
3,QCOM.OQ,Technology,"Qualcomm Inc, Ordinary Share, NASDAQ Stock Exc...",4295907706
4,KMPR.N,Financials,"Kemper Corp, Ordinary Share, New York Stock Ex...",4295908262
...,...,...,...,...
1005,NLY.N,Real Estate,"Annaly Capital Management Inc, Ordinary Share,...",4295903438
1006,MA.N,Technology,"Mastercard Inc, Ordinary Share, Class A, New Y...",4295902520
1007,BIIB.OQ,Healthcare,"Biogen Inc, Ordinary Share, NASDAQ Stock Excha...",4295906748
1008,SYF.N,Financials,"Synchrony Financial, Ordinary Share, New York ...",5042214022


### Getting the transcripts for our universe

Once we have our universe defined, let's request the transcripts data useing MTA API. The API allows single and multi-asset ingestion and offers document, sector, speaker and sentence level of request granularity. In the scope of our analysis, we will go for multi-asset sentence level request. Furthermore, we will benefit from the searching capabilities of the API and ask only for sentences containing word AI. More about the MTA API and it's capabilities can be found in the [official documentation](https://www.marketpsych.com/transcripts/guide/overview).


Before making our request though, below we provide a wrapper function that simplifies the process of fetching data from the API and by directly loading it into a pandas DataFrame.

In [6]:
def create_df(url: str) -> pd.DataFrame:
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Query failed with status code {response.status_code}: {response.json().get('error')}")
    data = response.json()['data']
    return pd.DataFrame(data)

In [7]:
def chunk_list(lst: list, chunk_size: int) -> list:
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

To begin, you'll need to replace APIKEY in the code snippet below with your API key provided by LSEG/MarketPsych. Further, we define the remaining request parameters and leverage the function above for the chunks of our universe. We devide our original universe into chunks to respect the API limits uning chunk_list function introduced below.

In [9]:
APIKEY = "" # Replace the following with your API key

chunk_size = 500
perm_ids_chunks = list(chunk_list(universe['IssuerOAPermID'], chunk_size))

start_date = '2016-01-01'
end_date = '2024-06-03'
search_word = 'AI'

events_df = pd.DataFrame()

for perm_ids in perm_ids_chunks:
    perm_ids_str = ','.join(perm_ids)
    url = f"https://dataapi.marketpsych.com/mta/data/{perm_ids_str}?apikey={APIKEY}&aggregation_type=sentence&start_date={start_date}&end_date={end_date}&event_type=ecc&dedup=True&search={search_word}&format=json"
    df = create_df(url)
    events_df = pd.concat([events_df, df])
events_df

,_id,document.id,document.status,document.last_edited,event.name,event.type,event.start_date,event.reference_date,event.host,event.host_permid,company.name,company.permid,company.ric,company.ticker,section.name,section.type,turn.idx,speaker.id,speaker.is_identified,speaker.first_name,speaker.last_name,speaker.company_name,speaker.position,speaker.is_organizer,speaker.other_name,sentence.idx,sentence.text,sentence.is_spam,analysis.ntokens,analysis.reference_time,analysis.keyphrases,analysis.idd_entities,analysis.entities,analysis.fin_sentiment,analysis.esg_sentiment,emotions.admiration,emotions.anger,emotions.annoyance,emotions.approval,emotions.confusion,emotions.disappointment,emotions.disapproval,emotions.fear,emotions.gratitude,emotions.optimism,emotions.pride,emotions.surprise,emotions.neutral,publ_time,search_matches,proc_time
0,65f5e52dc2cad1e9f691e7bfcde0c3c2_q-and-a_68_da...,65f5e52dc2cad1e9f691e7bfcde0c3c2,EDITED,2016-11-15T15:17:07,Q3 2016 Expedia Inc Earnings Call,ecc,2016-10-27T20:30:00,2016-09-30T00:00:00,Expedia Inc,4295899731,Expedia Group Inc,4295899731,EXPE.OQ,EXPE,q-and-a,report,68,darakhosrowshahi,True,Dara,Khosrowshahi,"Expedia, Inc.",President & CEO,True,,1,"Sure, as far as the opportunity and challenges...",None,58,None,{'TECHNOLOGY': 1},{'4295899731': 1},{'ORG_Expedia': 1},0.87,0.83,0.01,0.00,0.00,0.03,0.23,0.00,0.01,0.14,0.00,0.09,0.00,0.00,0.01,1477600200,[AI],NaN
1,65f5e52dc2cad1e9f691e7bfcde0c3c2_q-and-a_68_da...,65f5e52dc2cad1e9f691e7bfcde0c3c2,EDITED,2016-11-15T15:17:07,Q3 2016 Expedia Inc Earnings Call,ecc,2016-10-27T20:30:00,2016-09-30T00:00:00,Expedia Inc,4295899731,Expedia Group Inc,4295899731,EXPE.OQ,EXPE,q-and-a,report,68,darakhosrowshahi,True,Dara,Khosrowshahi,"Expedia, Inc.",President & CEO,True,,13,Those I think are experiences where AI can ser...,None,32,None,[],[],[],0.22,NaN,0.00,0.00,0.09,0.94,0.00,0.01,0.00,0.00,0.00,0.97,0.02,0.00,0.01,1477600200,[AI],NaN
2,65f5e52dc2cad1e9f691e7bfcde0c3c2_q-and-a_68_da...,65f5e52dc2cad1e9f691e7bfcde0c3c2,EDITED,2016-11-15T15:17:07,Q3 2016 Expedia Inc Earnings Call,ecc,2016-10-27T20:30:00,2016-09-30T00:00:00,Expedia Inc,4295899731,Expedia Group Inc,4295899731,EXPE.OQ,EXPE,q-and-a,report,68,darakhosrowshahi,True,Dara,Khosrowshahi,"Expedia, Inc.",President & CEO,True,,6,the more significant opportunity in AI near- t...,None,24,None,[],[],[],0.95,NaN,0.02,0.00,0.02,0.11,0.00,0.01,0.02,0.00,0.01,1.00,0.09,0.00,0.01,1477600200,[AI],NaN
3,ba0f6fe68a8446f8b46bf91269f4cbf8_q-and-a_38_br...,ba0f6fe68a8446f8b46bf91269f4cbf8,EDITED,2016-12-09T15:30:16Z,Q3 2016 Twilio Inc Earnings Call,ecc,2016-11-03T21:00:00,2016-09-30T00:00:00,Twilio Inc,4298095779,Twilio Inc,4298095779,TWLO.N,TWLO,q-and-a,report,38,bracelinbrent,True,Brent,Bracelin,Pacific Crest Securities,Analyst,False,,5,"And then, if you could just weigh a little bit...",None,25,None,[],[],[],-0.07,NaN,0.00,0.00,0.02,0.00,0.03,0.00,0.01,0.15,0.00,0.07,0.00,0.00,0.01,1478206800,[AI],NaN
4,ba0f6fe68a8446f8b46bf91269f4cbf8_q-and-a_38_br...,ba0f6fe68a8446f8b46bf91269f4cbf8,EDITED,2016-12-09T15:30:16Z,Q3 2016 Twilio Inc Earnings Call,ecc,2016-11-03T21:00:00,2016-09-30T00:00:00,Twilio Inc,4298095779,Twilio Inc,4298095779,TWLO.N,TWLO,q-and-a,report,38,bracelinbrent,True,Brent,Bracelin,Pacific Crest Securities,Analyst,False,,7,If you could just provide us a little more col...,None,30,None,{'ADOPTION': 1},[],[],0.12,NaN,0.01,0.00,0.00,0.02,0.10,0.00,0.00,0.04,0.02,0.06,0.00,0.00,0.00,1478206800,[AI],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16140,0a5d615bb8de9976d23874d45100265b_q-and-a_38_ja...,0a5d615bb8de9976d23874d45100265b,EDITED,2024-05-31T20:36:10Z,Q3 2024 Zscaler Inc Earnings Call,ecc,2024-05-30T20:30:00,2024-09-30T00:00:00,Zscaler Inc,5036689323,Zscaler Inc,5036689323,ZS.OQ,ZS.OQ,q-and-a,report,38,jagtarchaudhry,True,Jagtar,Chaudhry,Zscale

The resulting dataframe includes around 28,000 sentences from our universe, dating back to January 2016. It encompasses a broad range of emotion and sentiment metrics for each sentence. Additionally, each entry is provided with metadata related to the event, company, and speaker. A complete list of the output fields and their descriptions is available in the [documentation](https://www.marketpsych.com/transcripts/guide/fields). 

To make our final dataset ready for the analysis, we merge the transcripts data above with the universe data we obtained in the previous section and sort the values by the start date of the event.

In [10]:
events_df['company.permid'] = events_df['company.permid'].astype(str)
events_df = events_df.merge(universe, left_on = 'company.permid', right_on='IssuerOAPermID', how='inner')
events_df['start_date'] = pd.to_datetime(events_df['event.start_date']).dt.date
events_df = events_df.sort_values(by='start_date')
events_df

,_id,document.id,document.status,document.last_edited,event.name,event.type,event.start_date,event.reference_date,event.host,event.host_permid,company.name,company.permid,company.ric,company.ticker,section.name,section.type,turn.idx,speaker.id,speaker.is_identified,speaker.first_name,speaker.last_name,speaker.company_name,speaker.position,speaker.is_organizer,speaker.other_name,sentence.idx,sentence.text,sentence.is_spam,analysis.ntokens,analysis.reference_time,analysis.keyphrases,analysis.idd_entities,analysis.entities,analysis.fin_sentiment,analysis.esg_sentiment,emotions.admiration,emotions.anger,emotions.annoyance,emotions.approval,emotions.confusion,emotions.disappointment,emotions.disapproval,emotions.fear,emotions.gratitude,emotions.optimism,emotions.pride,emotions.surprise,emotions.neutral,publ_time,search_matches,proc_time,Instrument,TRBC Economic Sector Name,DocumentTitle,IssuerOAPermID,start_date
27814,27802443a70d9e0f251ef41cb1cc33bc_q-and-a_76_ha...,27802443a70d9e0f251ef41cb1cc33bc,EDITED,2016-02-08T18:33:03,Q4 2015 Celanese Corp Earnings Call,ecc,2016-01-22T15:00:00,2015-12-31T00:00:00,Celanese Corp,4295899696,Celanese Corp,4295899696,CE.N,CE,q-and-a,report,76,hassanahmed,True,Hassan,Ahmed,Alembic Global Advisors,Analyst,False,,2,Wanted to revisit the AI segment.,None,7,None,[],[],[],-0.01,NaN,0.01,0.00,0.0,0.01,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00,0.99,1453474800,[AI],NaN,CE.N,Basic Materials,"Celanese Corp, Ordinary Share, New York Stock ...",4295899696,2016-01-22
27812,27802443a70d9e0f251ef41cb1cc33bc_q-and-a_31_da...,27802443a70d9e0f251ef41cb1cc33bc,EDITED,2016-02-08T18:33:03,Q4 2015 Celanese Corp Earnings Call,ecc,2016-01-22T15:00:00,2015-12-31T00:00:00,Celanese Corp,4295899696,Celanese Corp,4295899696,CE.N,CE,q-and-a,report,31,davidbegleiter,True,David,Begleiter,Deutsche Bank,Analyst,False,,2,"Mark, in AI how should we think about the meth...",None,17,2015-01-22T00:00:00,[],{'COME_BIO': 1},"{'PERSON_Mark': 1, 'COMMODITY_ENM_methanol': 1}",-0.13,NaN,0.01,0.00,0.0,0.00,0.03,0.00,0.01,0.05,0.00,0.62,0.00,0.00,0.00,1453474800,[AI],NaN,CE.N,Basic Materials,"Celanese Corp, Ordinary Share, New York Stock ...",4295899696,2016-01-22
27813,27802443a70d9e0f251ef41cb1cc33bc_q-and-a_71_je...,27802443a70d9e0f251ef41cb1cc33bc,EDITED,2016-02-08T18:33:03,Q4 2015 Celanese Corp Earnings Call,ecc,2016-01-22T15:00:00,2015-12-31T00:00:00,Celanese Corp,4295899696,Celanese Corp,4295899696,CE.N,CE,q-and-a,report,71,jeffzekauskas,True,Jeff,Zekauskas,JPMorgan,Analyst,False,,1,"And then lastly with the AI results, do you th...",None,27,None,[],[],[],-0.15,NaN,0.01,0.00,0.0,0.14,0.01,0.00,0.00,0.00,0.01,0.98,0.00,0.00,0.00,1453474800,[AI],NaN,CE.N,Basic Materials,"Celanese Corp, Ordinary Share, New York Stock ...",4295899696,2016-01-22
27815,27802443a70d9e0f251ef41cb1cc33bc_q-and-a_76_ha...,27802443a70d9e0f251ef41cb1cc33bc,EDITED,2016-02-08T18:33:03,Q4 2015 Celanese Corp Earnings Call,ecc,2016-01-22T15:00:00,2015-12-31T00:00:00,Celanese Corp,4295899696,Celanese Corp,4295899696,CE.N,CE,q-and-a,report,76,hassanahmed,True,Hassan,Ahmed,Alembic Global Advisors,Analyst,False,,6,So the first part of the question is how much ...,None,21,2016-01-22T15:00:00,[],{'COME_BIO': 1},{'COMMODITY_ENM_methanol pricing': 1},-0.13,NaN,0.00,0.01,0.0,0.00,0.19,0.00,0.00,0.07,0.00,0.00,0.00,0.00,0.00,1453474800,[AI],NaN,CE.N,Basic Materials,"Celanese Corp, Ordinary Share, New York Stock ...",4295899696,2016-01-22
27816,27802443a70d9e0f251ef41cb1cc33bc_q-and-a_76_ha...,27802443a70d9e0f251ef41cb1cc33bc,EDITED,2016-02-08T18:33:03,Q4 2015 Celanese Corp Earnings Call,ecc,2016-01-22T15:00:00,2015-12-31T00:00:00,Celanese Corp,4295899696,Celanese Corp,4295899696,CE.N,CE,q-and-a,report,76,hassanahmed,True,Hassan,Ahmed,Alembic Global Advisors,Analyst,False,,7,Meaning with the ebbs and flows in methanol pr...,None,21,None,{'VOLATILITY': 1},{'COME_BIO': 1},{'COMMODITY_ENM_methanol pricing': 1},-0.90,NaN,0.00,0.00,0.0,0.00,0.11,0.00,0.00,0.95,0.00,0.00,0.00,0.00,0.00,1

## Section 2: Exploring AI Mentions trends across time and sectors
In this section we provide an analysis of AI mentions in earnings calls, focusing on both temporal and sectoral trends. With various plots we demonstrate how the frequency and emotions of AI-related sentences in the transcripts have evolved over time and how these patterns vary across different industries and companies. To scale the analysis we define reusable functions for grouping our initial dataset and producing the plots.

### Defining helper functions
First, we define a function to group our dataset. The ***group_dataset*** function below processes a DataFrame to perform aggregation and analysis based on specified columns passed as an input. 

In [11]:
def group_dataset(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    # Columns to average
    mean_columns = [
        'analysis.fin_sentiment',
        'analysis.esg_sentiment', 'emotions.admiration', 'emotions.anger',
        'emotions.annoyance', 'emotions.approval', 'emotions.confusion',
        'emotions.disappointment', 'emotions.disapproval', 'emotions.fear',
        'emotions.gratitude', 'emotions.optimism', 'emotions.pride',
        'emotions.surprise', 'emotions.neutral'
    ]

    # Aggregation functions for grouping
    aggregation_functions = {col: 'mean' for col in mean_columns}
    aggregation_functions['sentence.idx'] = 'count'
    aggregation_functions['company.permid'] = 'nunique'

    # Convert 'start_date' to datetime and create 'quarter' column
    df['start_date'] = pd.to_datetime(df['start_date'])
    df['quarter'] = df['start_date'].dt.to_period('Q')

    # Group by specified columns and aggregate
    grouped = df.groupby(cols).agg(aggregation_functions).reset_index()

    # Rename columns for clarity
    grouped.rename(columns={'sentence.idx': 'num_sentences', 'company.permid': 'company_count'}, inplace=True)

    # Sort by 'quarter' and reset index
    return grouped.sort_values(by='quarter').reset_index(drop=True)


Next, we define our plotting functions which will be used to produce different plots for provided metrics, sectors and companies. 

The function below produces a plot that visualizes changes in both the number of sentences and aggregated metrics, such as emotions and sentiments, across each quarter. It creates line plots for each metric provided in the input list, allowing for a detailed examination of how these metrics evolve over time. Additionally, it includes a bar plots to represent the number of sentences and the number of companies mentioning AI in their transcripts per quarter.

In [12]:
def plot_metric_and_sentence_trends(data: pd.DataFrame, metrics_to_plot: list, show_company_count: bool = False, name: str = 'all'):
    # Extract quarters as strings
    quarters = data['quarter'].astype(str)
    
    # Initialize subplot figure
    fig = make_subplots(rows=2, cols=1, 
                        shared_xaxes=False,
                        vertical_spacing=0.15,
                        row_width=[0.5, 1],
                        specs=[[{'secondary_y': True}], [{}]])

    # Add metrics traces
    for metric in metrics_to_plot:
        fig.add_trace(
            go.Scatter(x=quarters, y=data[metric], name=metric.split('.')[1], mode='lines+markers'),
            row=1, col=1, secondary_y=False
        )
    
    # Add sentence count trace
    fig.add_trace(
        go.Bar(x=quarters, y=data['num_sentences'], name='Sentence Count', opacity=0.6),
        row=1, col=1, secondary_y=True
    )
    fig.update_layout(yaxis_title='Emotion Score', height=800, title=f'Quarterly Emotion Trends for {name}')
    
    # Optionally add company count trace
    if show_company_count:
        fig.add_trace(
            go.Bar(x=quarters, y=data['company_count'], name='Company Count', marker_color='orange'),
            row=2, col=1
        )
        fig.update_yaxes(title_text="Sentence Count", secondary_y=True, row=1, col=1)
        fig.update_yaxes(title_text="Company Count", row=2, col=1)

    fig.show()


The function below generates two pie charts for visualizing the sectoral distribution of data. The first pie chart illustrates the distribution of the number of sentences across different economic sectors. The second pie chart represents the sectoral distribution of a specified emotion metric, allowing for the comparison of emotional content across sectors. 

In [13]:
def plot_sectoral_analysis(data: pd.DataFrame, metric: str):
    fig = make_subplots(rows=1, cols=2, specs=[[{'type':'pie'}, {'type':'pie'}]])

    fig.add_trace(
        go.Pie(labels=data['TRBC Economic Sector Name'], 
            values=data['num_sentences'],
            name='Number of sentences'),
        row=1, col=1
    )

    fig.add_trace(
        go.Pie(labels=data['TRBC Economic Sector Name'], 
            values=data[metric], 
            name=f"{metric.split('.')[1]}"),
        row=1, col=2
    )

    fig.update_layout(
        width=1000,
        height=500,
        annotations=[
            dict(text='Number of sentences', x=0.1, xref='paper', y=1.15, yref='paper', showarrow=False, font=dict(size=16)),
            dict(text=f"{metric.split('.')[1]}".capitalize(), x=0.8, xref='paper', y=1.15, yref='paper', showarrow=False, font=dict(size=16))
        ]
    )

    fig.show()


Finally the ***plot_sector_area_trend*** function generates an area chart that displays the historical trends of a specified metric across different economic sectors, organized by quarters througout the analyis period. While the previous function shows the aggregated sectoral distribution point in time, this function allows to explore how the sectoral distribution of the metric evolved over time.

In [14]:
def plot_sector_area_trend(data: pd.DataFrame, metric: str):
    data['quarter'] = data['quarter'].astype(str)
    fig = px.area(data, x = 'quarter',
                 y=metric, color="TRBC Economic Sector Name", width=1400, height=600)

    fig.update_layout(
        xaxis=dict(
            type='category',
            categoryorder='array',
        ),
        title = 'Number of constituents per sector historically',
        width=1000, height=500
    )
    fig.show()

### Aggregated view of the analysis

Below we group our initial dataset by quarter only, which will enable the aggregated view of the trends in AI mentions, related emotions and sentiments. We use previously defined ***group_dataset*** function as shown below:

In [15]:
group_cols = ['quarter']
events_by_quarter  = group_dataset(events_df, group_cols)
events_by_quarter

,quarter,analysis.fin_sentiment,analysis.esg_sentiment,emotions.admiration,emotions.anger,emotions.annoyance,emotions.approval,emotions.confusion,emotions.disappointment,emotions.disapproval,emotions.fear,emotions.gratitude,emotions.optimism,emotions.pride,emotions.surprise,emotions.neutral,num_sentences,company_count
0,2016Q1,0.317812,0.375000,0.022812,0.000937,0.020313,0.121875,0.014063,0.064375,0.003750,0.124687,0.002812,0.495937,0.049687,0.001250,0.146875,32,9
1,2016Q2,0.476316,0.702308,0.025526,0.003158,0.013158,0.099211,0.076316,0.072895,0.018947,0.038947,0.003684,0.433158,0.055000,0.001842,0.224737,38,8
2,2016Q3,0.391481,0.254167,0.073704,0.003704,0.019630,0.085926,0.037778,0.051111,0.021111,0.075926,0.003148,0.443148,0.065370,0.000926,0.202778,54,14
3,2016Q4,0.473617,0.535179,0.048582,0.009504,0.019220,0.144965,0.048723,0.035461,0.010142,0.047518,0.002908,0.416525,0.038511,0.007872,0.261348,141,28
4,2017Q1,0.593879,0.651379,0.055517,0.001638,0.014655,0.175690,0.020603,0.045345,0.008190,0.037672,0.003966,0.453362,0.072586,0.001207,0.244224,119,26
5,2017Q2,0.516276,0.563776,0.036327,0.003010,0.024286,0.158112,0.041276,0.025459,0.016888,0.033010,0.003469,0.451735,0.059031,0.004031,0.265816,197,38
6,2017Q3,0.613707,0.591743,0.056098,0.001024,0.010634,0.172537,0.045366,0.008634,0.025268,0.020000,0.006195,0.498878,0.059073,0.004634,0.265951,206,49
7,2017Q4,0.608773,0.616000,0.040409,0.002545,0.010500,0.172091,0.037409,0.010136,0.008455,0.024682,0.004455,0.531818,0.045955,0.001545,0.272091,222,47
8,2018Q1,0.643182,0.481111,0.035818,0.001273,0.008500,0.167591,0.041773,0.006273,0.010364,0.023773,0.005455,0.493909,0.057727,0.001364,0.310955,223,52
9,2018Q2,0.578213,0.624788,0.051348,0.001254,0.016489,0.180282,0.047524,0.006426,0.014984,0.024765,0.004702,0.463009,0.058746,0.003856,0.304859,319,72


As we now have the event dataset grouped by quarter, we pass it to ***plot_metric_and_sentence_trends*** function along with the metrics column to produce a visualisation showing the changes in AI mentions and emotions around it over time. We also define the metrics we are interested in below, which you can change with other metrics available in the grouped dataset.

In [16]:
metrics = [
       'emotions.optimism',
       'analysis.fin_sentiment','analysis.esg_sentiment'	
       ]

plot_metric_and_sentence_trends(events_by_quarter, metrics, show_company_count=True)

The graph above illustrates the changes in AI mentions and the associated optimism, financial, and ESG sentiments over time. It shows a gradual increase in the number of AI mentions, rising from just 32 in 2016 to 393 by the fourth quarter of 2022. However, following the public release of ChatGPT and it's advanced models, mentions surged dramatically, tripling to 3076 in the first quarter of 2023. This trend continued into the first quarter of 2024, with AI mentions reaching 5307, approximately 14 times the amount recorded in the fourth quarter of 2022. This growth can be attributed not only to the increased frequency of AI mentions but also to a rise in the number of companies focusing on AI, as indicated by the second bar plot. Interestingly, despite the heightened focus on AI, the graph reveals a decline in optimism, as well as a decrease in financial and ESG sentiments, following the rapid increase in AI-related discussions.

In regards to the latter, we also plot the fear and confusion emotions around AI mentions where we observe an increase after the popularization of AI, which might be perhaps explained by the realizations of the issues and the challanges of wider AI adoption.

In [17]:
metrics = [
           'emotions.confusion', 'emotions.fear',
       ]

plot_metric_and_sentence_trends(events_by_quarter, metrics)

### Sectoral view of the analysis
Next we dive into the sectoral analysis of the similar trends for which we use our ***group_dataset*** function and group our events data with the sector and the quarter as shown below.

In [18]:
group_cols = ['TRBC Economic Sector Name', 'quarter']
events_by_sector = group_dataset(events_df, group_cols)
events_by_sector

,TRBC Economic Sector Name,quarter,analysis.fin_sentiment,analysis.esg_sentiment,emotions.admiration,emotions.anger,emotions.annoyance,emotions.approval,emotions.confusion,emotions.disappointment,emotions.disapproval,emotions.fear,emotions.gratitude,emotions.optimism,emotions.pride,emotions.surprise,emotions.neutral,num_sentences,company_count
0,Basic Materials,2016Q1,-0.200000,NaN,0.004286,0.001429,0.064286,0.022857,0.052857,0.004286,0.005714,0.164286,0.001429,0.231429,0.000000,0.000000,0.141429,7,1
1,Consumer Non-Cyclicals,2016Q1,-0.067000,-0.010000,0.002000,0.000000,0.015000,0.080000,0.004000,0.183000,0.004000,0.280000,0.005000,0.420000,0.021000,0.000000,0.051000,10,2
2,Consumer Cyclicals,2016Q1,0.645000,0.000000,0.045000,0.005000,0.010000,0.125000,0.000000,0.090000,0.000000,0.000000,0.005000,0.325000,0.210000,0.020000,0.145000,2,2
3,Technology,2016Q1,0.842308,0.755000,0.045385,0.000769,0.002308,0.206923,0.003077,0.001538,0.003077,0.003077,0.001538,0.723077,0.073846,0.000000,0.223846,13,4
4,Consumer Non-Cyclicals,2016Q2,0.040000,NaN,0.000769,0.003077,0.024615,0.028462,0.089231,0.212308,0.025385,0.103077,0.001538,0.153077,0.003077,0.003077,0.103077,13,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,Financials,2024Q2,0.453689,0.572692,0.023883,0.000583,0.018835,0.207767,0.084563,0.004757,0.008738,0.047767,0.002913,0.423010,0.025146,0.003495,0.226699,106,25
271,Healthcare,2024Q2,0.639270,0.601486,0.045109,0.000292,0.006788,0.285985,0.020146,0.029343,0.006204,0.016788,0.003577,0.543066,0.058540,0.001971,0.273942,139,28
272,Industrials,2024Q2,0.642968,0.625967,0.033476,0.000668,0.014465,0.198824,0.043262,0.009786,0.014652,0.036283,0.005401,0.508904,0.050000,0.010936,0.227888,376,51
273,Technology,2024Q2,0.637548,0.656037,0.045100,0.001670,0.015141,0.218984,0.036424,0.013023,0.011523,0.032361,0.007490,0.523781,0.073822,0.003758,0.265029,3433,140


First, let's plot the sectoral distributaion of the frequency of AI mentions in the transcripts alongside with the distribution of the specific emotion or sentiment.

In [19]:
plot_sectoral_analysis(events_by_sector, 'emotions.optimism')

As expected, the technology sector is the primary driver of AI development and the most optimistic about it. Companies in consumer cyclical, healthcare, financials, and industrials are also highly optimistic, while the utilities and energy sectors are the least enthusiastic about AI. While you can experiment with various emotions and sentiments here, below we demonstarte an area plot to show how the emotions/sentiments across different sectors evolved over time.

In [20]:
plot_sector_area_trend(events_by_sector, 'emotions.optimism')

Finally, we select a sector and show the number of sentences, companies and emotional shifts within the sector utilizing the ***plot_metric_and_sentence_trends*** function.

In [21]:
sector = 'Financials'
data  = events_by_sector[events_by_sector['TRBC Economic Sector Name'] == sector]
metrics = [
       'emotions.optimism',
       'analysis.fin_sentiment','analysis.esg_sentiment']

plot_metric_and_sentence_trends(data, metrics, name= sector)

### Company level view of the analysis
Here, we show also the company level view of the analysis, first by grouping events dataset by Instrument and the quarter.

In [22]:
group_cols = ['Instrument', 'quarter', 'start_date']
events_by_company = group_dataset(events_df, group_cols)
events_by_company

,Instrument,quarter,start_date,analysis.fin_sentiment,analysis.esg_sentiment,emotions.admiration,emotions.anger,emotions.annoyance,emotions.approval,emotions.confusion,emotions.disappointment,emotions.disapproval,emotions.fear,emotions.gratitude,emotions.optimism,emotions.pride,emotions.surprise,emotions.neutral,num_sentences,company_count
0,MSFT.OQ,2016Q1,2016-01-28,0.930000,NaN,0.100000,0.000000,0.000000,0.100000,0.010000,0.000000,0.000000,0.000000,0.010000,0.960000,0.000000,0.00,0.060000,1,1
1,TSN.N,2016Q1,2016-02-05,0.190000,NaN,0.010000,0.000000,0.005000,0.020000,0.000000,0.005000,0.020000,0.255000,0.010000,0.865000,0.045000,0.00,0.000000,2,1
2,META.OQ,2016Q1,2016-01-27,0.764286,0.755,0.005714,0.001429,0.002857,0.367143,0.001429,0.002857,0.001429,0.000000,0.001429,0.605714,0.081429,0.00,0.262857,7,1
3,CE.N,2016Q1,2016-01-22,-0.200000,NaN,0.004286,0.001429,0.064286,0.022857,0.052857,0.004286,0.005714,0.164286,0.001429,0.231429,0.000000,0.00,0.141429,7,1
4,MAT.OQ,2016Q1,2016-02-01,0.920000,NaN,0.080000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.640000,0.410000,0.03,0.090000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3690,RSG.N,2024Q2,2024-04-30,0.790000,NaN,0.000000,0.000000,0.000000,0.030000,0.070000,0.010000,0.410000,0.070000,0.000000,0.120000,0.000000,0.00,0.010000,1,1
3691,RVTY.N,2024Q2,2024-04-29,0.950000,NaN,0.020000,0.000000,0.000000,0.060000,0.000000,0.000000,0.000000,0.030000,0.000000,1.000000,0.000000,0.00,0.000000,1,1
3692,DGX.N,2024Q2,2024-04-23,0.594000,0.774,0.016000,0.000000,0.004000,0.616000,0.006000,0.004000,0.002000,0.022000,0.006000,0.814000,0.018000,0.00,0.102000,5,1
3693,LMT.N,2024Q2,2024-04-23,0.215000,0.520,0.045000,0.000000,0.020000,0.377500,0.010000,0.242500,0.005000,0.062500,0.062500,0.525000,0.002500,0.00,0.005000,4,1


First, let's look into the some of the emotions and sentiments for the top 10 companies by the number of AI mentions in the first quarter of 20224.

In [23]:
filtered_df = events_by_company[events_by_company['quarter'] == '2024Q1']
top_companies = filtered_df.nlargest(10, 'num_sentences').set_index('Instrument')
emotions = ['emotions.optimism','analysis.fin_sentiment','analysis.esg_sentiment']
heatmap_data = top_companies[emotions].reset_index()

fig = go.Figure(data=go.Heatmap(
    z=heatmap_data[emotions].values,
    x=emotions,
    y=heatmap_data['Instrument'],
    colorscale='Viridis')
    )

fig.update_layout(title='Emotion Comparison for Q1 2024 for top 10 companies by AI mentions')
fig.show()

According to the graph, Salesforce (CRM.N) has the most AI mentions during 2024Q1, however, it is not the most optimistic company around AI amongst the top 10. Below we demonstrate also how the nmber of mentions, emotions and sentiments changed over time for Salesforce. Please note, that the metrics can be updated if interested in the trends of the rest of the emotions, sentiments and the companies.

In [24]:
company = 'CRM.N'
data  = events_by_company[events_by_company['Instrument'] == company]
metrics = [       
    'emotions.optimism',
    'analysis.fin_sentiment','analysis.esg_sentiment'
    ]

plot_metric_and_sentence_trends(data, metrics, name = company)

## Section 3: Constructing Thematic Portfolios based on the AI mentions
In this section, we will construct portfolios and track their performance over the analysis period from the first quarter of 2016 to the second quarter of 2024. We employ two approaches for portfolio construction:

*   **Clustering Approach:** In this method, we divide our universe into two clusters based on the number of mentions, emotions, and sentiments.
*   **Quantile-Based Portfolios:** Here, we categorize companies into two quantiles based on a specific metric, which in our case is the number of sentences.

Refer to the relevant sub-section for detailed methodology.

### Getting the pricing data and calculating the returns
We use LD libraries for Python to get the historical close prices for our universe for the analysis period. To track the returns, we calculate the two-month returns of each stock in the portfolios following the event start date. The two-month window is chosen to account for the fact that companies may make announcements on different days throughout the quarter. This approach helps avoid including returns that occur after the subsequent earnings call event date.

Below we find the unique instruments from our event dataset and request close prices for January 2016 to July 2024.

In [25]:
rics  = events_by_company['Instrument'].drop_duplicates().to_list()
prices  = ld.get_history(rics, 'TRDPRC_1', start='2016-01-01', end='2024-07-30')

Next, we define the **calculate_period_returns** function which calculates the percentage returns for stocks over a specified period following an event date.

In [26]:
def calculate_period_returns(event_df: pd.DataFrame, prices_df: pd.DataFrame) -> pd.DataFrame:
    # define helper to calculate percentage change in series
    def calculate_pct_change(series):
        return (series.iloc[-1] - series.iloc[0]) / series.iloc[0] * 100

    returns = []
    end_dates = []
    for row in event_df.itertuples():
        # Define start and end dates for the period
        start = pd.to_datetime(row.start_date) + timedelta(days=1)
        end = start + relativedelta(months=2)
        ric = row.Instrument
        # Filter prices for the given period
        prices_ric = prices_df[ric][(prices_df[ric].index > start) & (prices_df[ric].index <= end)]
        # Calculate percentage change and store results
        returns.append(calculate_pct_change(prices_ric))
        end_dates.append(end)
    
    # Add returns and end dates to event_df
    event_df['returns'] = returns
    event_df['end_date'] = end_dates
    return event_df


events_by_company = calculate_period_returns(events_by_company, prices)
events_by_company

,Instrument,quarter,start_date,analysis.fin_sentiment,analysis.esg_sentiment,emotions.admiration,emotions.anger,emotions.annoyance,emotions.approval,emotions.confusion,emotions.disappointment,emotions.disapproval,emotions.fear,emotions.gratitude,emotions.optimism,emotions.pride,emotions.surprise,emotions.neutral,num_sentences,company_count,returns,end_date
0,MSFT.OQ,2016Q1,2016-01-28,0.930000,NaN,0.100000,0.000000,0.000000,0.100000,0.010000,0.000000,0.000000,0.000000,0.010000,0.960000,0.000000,0.00,0.060000,1,1,0.0,2016-03-29
1,TSN.N,2016Q1,2016-02-05,0.190000,NaN,0.010000,0.000000,0.005000,0.020000,0.000000,0.005000,0.020000,0.255000,0.010000,0.865000,0.045000,0.00,0.000000,2,1,13.098404,2016-04-06
2,META.OQ,2016Q1,2016-01-27,0.764286,0.755,0.005714,0.001429,0.002857,0.367143,0.001429,0.002857,0.001429,0.000000,0.001429,0.605714,0.081429,0.00,0.262857,7,1,1.318956,2016-03-28
3,CE.N,2016Q1,2016-01-22,-0.200000,NaN,0.004286,0.001429,0.064286,0.022857,0.052857,0.004286,0.005714,0.164286,0.001429,0.231429,0.000000,0.00,0.141429,7,1,9.557344,2016-03-23
4,MAT.OQ,2016Q1,2016-02-01,0.920000,NaN,0.080000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.640000,0.410000,0.03,0.090000,1,1,7.871537,2016-04-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3690,RSG.N,2024Q2,2024-04-30,0.790000,NaN,0.000000,0.000000,0.000000,0.030000,0.070000,0.010000,0.410000,0.070000,0.000000,0.120000,0.000000,0.00,0.010000,1,1,3.454194,2024-07-01
3691,RVTY.N,2024Q2,2024-04-29,0.950000,NaN,0.020000,0.000000,0.000000,0.060000,0.000000,0.000000,0.000000,0.030000,0.000000,1.000000,0.000000,0.00,0.000000,1,1,2.202729,2024-06-30
3692,DGX.N,2024Q2,2024-04-23,0.594000,0.774,0.016000,0.000000,0.004000,0.616000,0.006000,0.004000,0.002000,0.022000,0.006000,0.814000,0.018000,0.00,0.102000,5,1,3.551913,2024-06-24
3693,LMT.N,2024Q2,2024-04-23,0.215000,0.520,0.045000,0.000000,0.020000,0.377500,0.010000,0.242500,0.005000,0.062500,0.062500,0.525000,0.002500,0.00,0.005000,4,1,1.447997,2024-06-24


### Portfolio Construction Approach 1: Clusttering our universe
At this step, we will cluster our universe into two groups using the KMeans algorithm, based on mentions, sentiments, and emotions as input features. The clustering is performed separately for each quarter to prevent look-ahead bias, ensuring that only data available up to that point in time is used for clustering.

The **cluster_companies** function below groups companies by quarter and then clusters them based on the provided features.

In [27]:
def cluster_companies(df:pd.DataFrame, features:list) ->tuple[pd.DataFrame, list]:
    def cluster_group(group):
        # Fill missing values (kmeans can't handle NaNs) and scale features
        for col in features:
            group[col] = group[col].fillna(0)
        scaler = StandardScaler()
        scaled_features = scaler.fit_transform(group[features])
        
        # Apply KMeans clustering
        kmeans = KMeans(n_clusters=2, random_state=42)
        group['cluster'] = kmeans.fit_predict(scaled_features)
        return group, kmeans.cluster_centers_

    clustered_df = pd.DataFrame()
    cluster_centers = []

    # Group by quarter and apply clustering
    for quarter, group in df.groupby('quarter'):
        clustered_group, centers = cluster_group(group)
        clustered_df = pd.concat([clustered_df, clustered_group])
        cluster_centers.append({quarter: centers})

    return clustered_df, cluster_centers


In [28]:
features = [
        'analysis.fin_sentiment', 'analysis.esg_sentiment','emotions.admiration', 
        'emotions.anger', 'emotions.annoyance', 'emotions.approval', 
        'emotions.confusion', 'emotions.disappointment', 'emotions.disapproval', 
        'emotions.fear', 'emotions.gratitude', 'emotions.optimism', 
        'emotions.pride', 'emotions.surprise', 'emotions.neutral', 
        'num_sentences'
]

events_by_company, cluster_means = cluster_companies(events_by_company, features)
events_by_company

,Instrument,quarter,start_date,analysis.fin_sentiment,analysis.esg_sentiment,emotions.admiration,emotions.anger,emotions.annoyance,emotions.approval,emotions.confusion,emotions.disappointment,emotions.disapproval,emotions.fear,emotions.gratitude,emotions.optimism,emotions.pride,emotions.surprise,emotions.neutral,num_sentences,company_count,returns,end_date,cluster
0,MSFT.OQ,2016Q1,2016-01-28,0.930000,0.000,0.100000,0.000000,0.000000,0.100000,0.010000,0.000000,0.000000,0.000000,0.010000,0.960000,0.000000,0.00,0.060000,1,1,0.0,2016-03-29,1
1,TSN.N,2016Q1,2016-02-05,0.190000,0.000,0.010000,0.000000,0.005000,0.020000,0.000000,0.005000,0.020000,0.255000,0.010000,0.865000,0.045000,0.00,0.000000,2,1,13.098404,2016-04-06,1
2,META.OQ,2016Q1,2016-01-27,0.764286,0.755,0.005714,0.001429,0.002857,0.367143,0.001429,0.002857,0.001429,0.000000,0.001429,0.605714,0.081429,0.00,0.262857,7,1,1.318956,2016-03-28,1
3,CE.N,2016Q1,2016-01-22,-0.200000,0.000,0.004286,0.001429,0.064286,0.022857,0.052857,0.004286,0.005714,0.164286,0.001429,0.231429,0.000000,0.00,0.141429,7,1,9.557344,2016-03-23,0
4,MAT.OQ,2016Q1,2016-02-01,0.920000,0.000,0.080000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.640000,0.410000,0.03,0.090000,1,1,7.871537,2016-04-02,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3690,RSG.N,2024Q2,2024-04-30,0.790000,0.000,0.000000,0.000000,0.000000,0.030000,0.070000,0.010000,0.410000,0.070000,0.000000,0.120000,0.000000,0.00,0.010000,1,1,3.454194,2024-07-01,1
3691,RVTY.N,2024Q2,2024-04-29,0.950000,0.000,0.020000,0.000000,0.000000,0.060000,0.000000,0.000000,0.000000,0.030000,0.000000,1.000000,0.000000,0.00,0.000000,1,1,2.202729,2024-06-30,0
3692,DGX.N,2024Q2,2024-04-23,0.594000,0.774,0.016000,0.000000,0.004000,0.616000,0.006000,0.004000,0.002000,0.022000,0.006000,0.814000,0.018000,0.00,0.102000,5,1,3.551913,2024-06-24,0
3693,LMT.N,2024Q2,2024-04-23,0.215000,0.520,0.045000,0.000000,0.020000,0.377500,0.010000,0.242500,0.005000,0.062500,0.062500,0.525000,0.002500,0.00,0.005000,4,1,1.447997,2024-06-24,0


Below we plot the cluster centers for the last quarter of the analysis to identify the cluster patterns.

In [29]:
fig = go.Figure(data=[
    go.Bar(name='Cluster 0', x=features, y=list(cluster_means[-1].values())[0][0]),
    go.Bar(name='Cluster 1', x=features, y=list(cluster_means[-1].values())[0][1])
])

fig.update_layout(barmode='group', title='Cluster means',
                  xaxis_title='Features', yaxis_title='Values')

fig.show()

As shown in the plot above, Cluster 0 has higher means for positive sentiments, emotions, and the number of mentions, while showing lower means for negative emotions. Therefore, we will define Cluster 0 as the "higher mean" cluster and Cluster 1 as the "lower mean" cluster for the subsequent portfolio analysis.

### Portfolio Construction Approach 2: Assigning quantiles to the Universe
In this step, we will devide our quarterly universe into 2 quantiles based on a metric, e.g the number of sentences.

Below, we first define our metric and use ***compute_quantiles_per_quarter*** function which categorizes data into quantiles for each quarter based on a specified metric column. 

In [30]:
metric = 'num_sentences'

In [31]:
def compute_quantiles_per_quarter(df: pd.DataFrame, sentiment_column: str) -> pd.DataFrame:
    def preprocess_sentiment(x):
        # Add a small random noise to avoid ties in quantile calculation
        noise = np.random.normal(0, 1e-9, len(x))
        return x + noise
    
    def calculate_quartiles(x):
        try:
            x = preprocess_sentiment(x)
            # Compute quartiles
            return pd.qcut(x, 2, labels=False) + 1
        except ValueError:
            # Handle cases with insufficient data
            return pd.Series(np.nan, index=x.index)
    
    # Apply quantile calculation within each quarter
    df['quantile'] = df.groupby('quarter')[sentiment_column].transform(calculate_quartiles)
    
    return df

events_by_company = compute_quantiles_per_quarter(events_by_company, metric)
events_by_company

,Instrument,quarter,start_date,analysis.fin_sentiment,analysis.esg_sentiment,emotions.admiration,emotions.anger,emotions.annoyance,emotions.approval,emotions.confusion,emotions.disappointment,emotions.disapproval,emotions.fear,emotions.gratitude,emotions.optimism,emotions.pride,emotions.surprise,emotions.neutral,num_sentences,company_count,returns,end_date,cluster,quantile
0,MSFT.OQ,2016Q1,2016-01-28,0.930000,0.000,0.100000,0.000000,0.000000,0.100000,0.010000,0.000000,0.000000,0.000000,0.010000,0.960000,0.000000,0.00,0.060000,1,1,0.0,2016-03-29,1,1
1,TSN.N,2016Q1,2016-02-05,0.190000,0.000,0.010000,0.000000,0.005000,0.020000,0.000000,0.005000,0.020000,0.255000,0.010000,0.865000,0.045000,0.00,0.000000,2,1,13.098404,2016-04-06,1,1
2,META.OQ,2016Q1,2016-01-27,0.764286,0.755,0.005714,0.001429,0.002857,0.367143,0.001429,0.002857,0.001429,0.000000,0.001429,0.605714,0.081429,0.00,0.262857,7,1,1.318956,2016-03-28,1,2
3,CE.N,2016Q1,2016-01-22,-0.200000,0.000,0.004286,0.001429,0.064286,0.022857,0.052857,0.004286,0.005714,0.164286,0.001429,0.231429,0.000000,0.00,0.141429,7,1,9.557344,2016-03-23,0,2
4,MAT.OQ,2016Q1,2016-02-01,0.920000,0.000,0.080000,0.010000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.640000,0.410000,0.03,0.090000,1,1,7.871537,2016-04-02,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3690,RSG.N,2024Q2,2024-04-30,0.790000,0.000,0.000000,0.000000,0.000000,0.030000,0.070000,0.010000,0.410000,0.070000,0.000000,0.120000,0.000000,0.00,0.010000,1,1,3.454194,2024-07-01,1,1
3691,RVTY.N,2024Q2,2024-04-29,0.950000,0.000,0.020000,0.000000,0.000000,0.060000,0.000000,0.000000,0.000000,0.030000,0.000000,1.000000,0.000000,0.00,0.000000,1,1,2.202729,2024-06-30,0,1
3692,DGX.N,2024Q2,2024-04-23,0.594000,0.774,0.016000,0.000000,0.004000,0.616000,0.006000,0.004000,0.002000,0.022000,0.006000,0.814000,0.018000,0.00,0.102000,5,1,3.551913,2024-06-24,0,1
3693,LMT.N,2024Q2,2024-04-23,0.215000,0.520,0.045000,0.000000,0.020000,0.377500,0.010000,0.242500,0.005000,0.062500,0.062500,0.525000,0.002500,0.00,0.005000,4,1,1.447997,2024-06-24,0,1


### Calculating and visualizing the portfolio returns

As already mentioned, we employ two approaches for portfolio construction: one based on quantiles and the other based on clusters. For each approach, we create both equal-weighted and weighted portfolios based on the provided metric. 

The function below calculates and adds weights for each universe per quarter. Note that it requires the event DataFrame, the metric, and the portfolio type as inputs, with the portfolio type being crucial for correct grouping to calculate the weights.

In [32]:
def calculate_weights(df: pd.DataFrame, metric:str = 'num_sentences', portf_type:str = 'quantile')-> pd.DataFrame:
    # Normalize sentiment values if there are any negative values
    if (df[metric] < 0).any():
        scaler = MinMaxScaler()
        df['normalized_sentiment'] = scaler.fit_transform(np.array(df[metric]).reshape(-1, 1))
        metric = 'normalized_sentiment'
    
    # Calculate total sentiment value for each (quarter, cluster) combination
    total_cluster_value = df.groupby(['quarter', portf_type])[metric].transform('sum')
    
    # Compute weights as the ratio of individual sentiment to total cluster sentiment
    df['weight'] = df[metric] / total_cluster_value
    return df

Now, that we have all the necessary components for constructing, rebalancing, and calculating the returns of our portfolios, we define the ***rebalance_portfolio_and_calculate_returns*** function. As the name suggests, this function constructs, rebalances, and calculates portfolios based on the specified portfolio types. It returns detailed portfolio rebalancing results, including information on portfolio assets and returns for each quarter, as well as summary results with only the total number of assets and aggregated returns for each quarter.

In [33]:
def rebalance_portfolio_and_calculate_returns(df: pd.DataFrame, portf_type:str = 'quantile')-> tuple[dict, dict]:
    # Drop rows with missing values in the specified columns
    df = df.dropna(subset=portf_type).dropna(subset='returns')
    
    # Initialize dictionaries for detailed and summary portfolio data
    portf_detailed = {}
    portf_summary = {}
    # Map portfolio types to descriptive names
    portf_name_dict ={'quantile': {1:'lowest quantile', 2:'highest quantile'},
                     'cluster': {0: 'higher mean', 1: 'lower mean'}}
    
    # Process each unique portfolio type
    for portf_name in sorted(df[portf_type].unique()):
        portf_data= df[df[portf_type] == portf_name]
        port_name_key = portf_name_dict[portf_type][portf_name]
        portf_detailed[port_name_key] = []
        portf_summary[port_name_key] = []
        portf_summary['weighted'] = []

        # Process each unique quarter
        for quarter in portf_data['quarter'].astype(str).unique():
            quarter_data = portf_data[portf_data['quarter'] == quarter]
            assets = quarter_data['Instrument'].tolist()
            returns = quarter_data['returns'].tolist()

            # Calculate weighted average returns if portfolio type is 'quantile' and add to the summary dict. Cluster based approach is equal-weighted only
            # if portf_type == 'quantile':
            weights = quarter_data['weight'].tolist()
            weighted_avg_returns = np.dot(returns, weights)
            portf_summary['weighted'].append({
                quarter: {
                    'return': weighted_avg_returns,
                    'n_assets': len(df[df['quarter'] == quarter]['Instrument'].unique())
                }
            })

            # Calculate average returns
            avg_return = np.mean(returns)

            # Store detailed and summary metrics
            portf_detailed[port_name_key].append({
                quarter: {
                    'assets': assets,
                    'returns': returns,
                    'avg_return': avg_return
                }
            })
            portf_summary[port_name_key].append({
                quarter: {
                    'return': avg_return,
                    'n_assets': len(assets)
                }
            })
    return portf_detailed, portf_summary



So, let's now add the weights to our dataset and run the rebalance_portfolio_and_calculate_returns function below for cluster based approach.

In [34]:
portf_type = 'cluster'
events_by_company = calculate_weights(events_by_company, metric, portf_type)
portfolio_detailed_view, portfolio_summary_view = rebalance_portfolio_and_calculate_returns(events_by_company, portf_type)
print('portfolio_names:', list(portfolio_summary_view.keys()))
portfolio_summary_view

portfolio_names: ['higher mean', 'weighted', 'lower mean']


{'higher mean': [{'2016Q1': {'return': 9.557344064386323, 'n_assets': 1}},
  {'2016Q2': {'return': 6.251691495663083, 'n_assets': 4}},
  {'2016Q3': {'return': -2.8189479392783094, 'n_assets': 7}},
  {'2016Q4': {'return': 5.7607139574025155, 'n_assets': 11}},
  {'2017Q1': {'return': -1.667404459804022, 'n_assets': 7}},
  {'2017Q2': {'return': 1.0206453400895847, 'n_assets': 16}},
  {'2017Q3': {'return': 6.995315157260509, 'n_assets': 25}},
  {'2017Q4': {'return': 2.5346743756131493, 'n_assets': 20}},
  {'2018Q1': {'return': -1.2209338774242198, 'n_assets': 50}},
  {'2018Q2': {'return': 2.287648159931029, 'n_assets': 9}},
  {'2018Q3': {'return': -3.732801460371706, 'n_assets': 65}},
  {'2018Q4': {'return': -5.690908106534571, 'n_assets': 3}},
  {'2019Q1': {'return': 6.654875820628354, 'n_assets': 29}},
  {'2019Q2': {'return': -1.637279427507174, 'n_assets': 8}},
  {'2019Q3': {'return': 0.6084732097130838, 'n_assets': 46}},
  {'2019Q4': {'return': 5.539713900041456, 'n_assets': 31}},
  {'

The dictionary above contains information about the returns and number of assets per quarter for each portfolio. Below, we define ***summarize_results_by*** function, which transforms the portfolio_summary_view dictionary into a dataframe for subsequent visualisation of the results.

In [35]:
def summarize_results_by(portfolio_summary:dict, port_name:str, by:str) -> pd.DataFrame:
    df = pd.DataFrame()
    # Iterate through each key in the portfolio summary
    for key in portfolio_summary.keys():
        key_df = pd.DataFrame(portfolio_summary[key])
        
        # Extract diagonal values and the relevant data
        diagonal_values = np.diag(key_df)
        data = [ret[by] for ret in diagonal_values]
        
        # Create a new DataFrame with extracted data
        new_df = pd.DataFrame([data], columns=key_df.columns)
        new_df[port_name] = key
        
        # Concatenate new data to the main DataFrame
        df = pd.concat([df, new_df])
    
    # Set the specified portfolio name as the index
    df = df.set_index(port_name)
    
    return df

Further, we run this function for the number of assets and returns (with cumulative sums applied to returns) to have the respective dataframes

In [36]:
assets = summarize_results_by(portfolio_summary_view, portf_type, 'n_assets').T
rets =  summarize_results_by(portfolio_summary_view, portf_type, 'return').T.cumsum()
rets

cluster,higher mean,weighted,lower mean
2016Q1,9.557344,6.278153,5.781012
2016Q2,15.809036,17.450685,10.238474
2016Q3,12.990088,21.460063,12.737030
2016Q4,18.750802,33.818233,16.643530
2017Q1,17.083397,33.576627,18.092009
2017Q2,18.104042,41.060926,20.480727
2017Q3,25.099358,45.665161,22.473488
2017Q4,27.634032,48.774941,25.532118
2018Q1,26.413098,59.791843,36.549019
2018Q2,28.700746,64.347684,39.686359


Finally, we show the cumulative returns along with the number of assets per portfolio over the analysis period.

In [37]:
def visulize_results(rets_df:pd.DataFrame, assets_df:pd.DataFrame):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    for col in rets_df.columns:
        fig.add_trace(
            go.Scatter(x=rets_df.index, y=rets_df[col].dropna(), mode='lines', name=f"{col} - return"),
            secondary_y=True,
        )

    for col in assets_df.columns.drop('weighted'):
        fig.add_trace(
            go.Bar(x=assets_df.index, y=assets_df[col], name=f"{col} - n_assets"),
            secondary_y=False,
        )

    fig.update_layout(
        title_text="Portfolio structure and returns",
        xaxis_title="Quarters",
        yaxis_title="Number of assets",
        legend_title="Features"
    )

    fig.update_yaxes(title_text="Number of assets", secondary_y=False)
    fig.update_yaxes(title_text="Cumulaive returns", secondary_y=True)

    fig.show()
visulize_results(rets, assets)

The graph above illustrates the number of assets and returns for each portfolio. The highest cumulative returns are observed in the weighted portfolio, followed by the higher mean portfolio. The lower mean portfolio, which includes companies with lower positive and higher negative emotions, has the lowest returns. Regarding asset allocation, the number of assets in each cluster varies significantly from quarter to quarter. 

Below, we will examine the results for quantile-based portfolios.

In [38]:
portf_type = 'quantile'
events_by_company = calculate_weights(events_by_company, metric, portf_type)
portfolio_detailed_view, portfolio_summary_view = rebalance_portfolio_and_calculate_returns(events_by_company, portf_type)
assets = summarize_results_by(portfolio_summary_view, portf_type, 'n_assets').T
rets =  summarize_results_by(portfolio_summary_view, portf_type, 'return').T.cumsum()
visulize_results(rets, assets)

The cumulative portfolio returns generally align with the clustering-based approach, with the weighted portfolio having the largest return. The gap between the weighted portfolio and the highest quantiles is significant. Additionally, both the weighted and highest return portfolios outperform the cluster-based weighted and higher mean results. In terms of the number of assets, as expected, the highest and lowest quantile portfolios have an almost equal distribution.

## Final remarks

The article demonstrated how  LSEG MarketPsych Transcripts Analytics (MTA) API can be used to ingest sentences with AI mentions from corporate transcripts and perform exploratory and portfolio construction analysis. The main goal was to examine a. how the frequency and sentiment of AI mentions in earnings calls have evolved over time and per sector and b. to assess whether these patterns can be leveraged to build effective thematic portfolios.

The methodologies and examples presented are primarily intended for educational and exploratory purposes. Although we observed positive results for the constructed thematic portfolios—where portfolios comprised of companies with frequent AI mentions and overall positive sentiments outperformed those with lower rankings—these findings should be considered preliminary insights. They serve as a foundation for further investigation rather than conclusive evidence. Users are encouraged to experiment with different data universes, search terms, and portfolio construction strategies.